In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
#tf.enable_v2_behavior()
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 0-255の値が入っているので、0-1に収まるよう正規化します
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [4]:
x_vec = []
for i in range(x_train.shape[0]):
    x_vec_ = np.concatenate([x_train[i][j, ::-2*(j%2)+1] for j in range(x_train.shape[1])])
    x_vec.append(x_vec_)
x_train_1d = np.vstack(x_vec)

x_vec = []
for i in range(x_test.shape[0]):
    x_vec_ = np.concatenate([x_test[i][j, ::-2*(j%2)+1] for j in range(x_test.shape[1])])
    x_vec.append(x_vec_)
x_test_1d = np.vstack(x_vec)

In [5]:
# Retrieving a component

def block(*dimensions, norm = 1):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return np.random.normal(loc = 1/np.max(size), scale = 1/np.max(size), size = size)

def create_MPS(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    mps = [
        tn.Node( block(dim, bond_dim) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(rank-2)] + \
        [tn.Node( block(bond_dim, dim) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank-1):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

def create_MPS_labeled(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    norm = 1 / bond_dim
    mps = [
        tn.Node( block(dim, bond_dim, norm=norm) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim, norm = norm)) for _ in range(half)] + \
        [tn.Node( block(bond_dim, label_dim, bond_dim, norm=norm) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim, norm=norm)) for _ in range(half, rank-2)] + \
        [tn.Node( block(bond_dim, dim, norm=norm) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges


In [6]:
def feature_map(p):
    phi = [1-p, p]
    return phi

def data_tensorize(vec):
    data_tensor = [tn.Node(feature_map(p)) for p in vec]
    return data_tensor

In [34]:
label_len = 1
label_dim = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10
# mps, edges = create_MPS(rank, dim, bond_dim)
mps, edges = create_MPS_labeled(rank, dim, bond_dim)

test_vec = x_train_1d[0]
data_tensor = data_tensorize(test_vec)

edges.append(data_tensor[0][0] ^ mps[0][0])
half_len = np.int(len(data_tensor) / 2)
[edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
[edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) for i in range(half_len + label_len, data_len + label_len)]
for k in reversed(range(len(edges))):
    A = tn.contract(edges[k])
result = A.tensor

In [35]:
print(A.tensor.numpy())
print(A.tensor.numpy().astype("float32"))
print(tf.math.log(A.tensor).numpy())
print(tf.nn.softmax(A.tensor).numpy())

[0.00721898 0.00809315 0.00991432 0.00682729 0.00746707 0.00656614
 0.00692087 0.0070252  0.00647266 0.00567744]
[0.00721898 0.00809315 0.00991432 0.00682729 0.00746707 0.00656614
 0.00692087 0.0070252  0.00647266 0.00567744]
[-4.93104215 -4.81673698 -4.61377472 -4.98682696 -4.8972527  -5.02582857
 -4.97321438 -4.95825138 -5.04016787 -5.17125548]
[0.10000001 0.10008746 0.10026991 0.09996085 0.10002482 0.09993475
 0.0999702  0.09998063 0.0999254  0.09984597]


In [7]:
def block(*dimensions):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return tf.Variable(
        tf.random.normal(shape=size, dtype=tf.dtypes.float64, mean= 1/np.max(size), stddev = 1/np.max(size)),
        trainable=True)

def create_blocks(rank, dim, bond_dim, label_dim):
    half = np.int((rank - 2) / 2)
    blocks = [
        block(dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half)] + \
        [ block(bond_dim, label_dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half, rank-2)] + \
        [ block(bond_dim, dim) 
        ]
    return blocks

def create_MPS_labeled(blocks, rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    mps = []
    for b in blocks:
        mps.append(tn.Node(b))

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

In [8]:
class TNLayer(tf.keras.layers.Layer):
    def __init__(self, input_len, label_num, bond_dim):
        self.label_len = 1
        self.label_dim = label_num
        self.rank = input_len
        self.dim = 2
        self.bond_dim = bond_dim
        #super(TNLayer, self).__init__()
        super().__init__()
        # Create the variables for the layer.
        self.blocks = create_blocks(self.rank, self.dim, self.bond_dim, self.label_dim)

    def call(self, inputs):
        def f(input_vec, blocks, rank, dim, bond_dim, label_len):
            mps, edges = create_MPS_labeled(blocks, rank, dim, bond_dim)
            data_tensor = []
            for p in tf.unstack(input_vec):
                data_tensor.append(tn.Node([1-p, p]))
            edges.append(data_tensor[0][0] ^ mps[0][0])
            half_len = np.int(rank / 2)
            [edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
            [edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) \
                 for i in range(half_len + label_len, rank + label_len)]
            for k in reversed(range(len(edges))):
                A = tn.contract(edges[k])
            #result = tf.math.log(A.tensor)
            result = A.tensor# - tf.math.reduce_max(A.tensor)
            return result

        result = tf.vectorized_map(
        lambda vec: f(vec, self.blocks, self.rank, self.dim, self.bond_dim, self.label_len), inputs)
        return result

In [40]:
N = x_train_1d.shape[1]
label_len = 1
label_num = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10

tf.keras.backend.set_floatx('float64')

tn_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(N,)),
        TNLayer(N, label_num, bond_dim),
        tf.keras.layers.Softmax()
    ])

tn_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tn_layer_6 (TNLayer)         (None, 10)                157440    
_________________________________________________________________
softmax_6 (Softmax)          (None, 10)                0         
Total params: 157,440
Trainable params: 157,440
Non-trainable params: 0
_________________________________________________________________


In [41]:
def step_decay(epoch):
    x = 1e-4
    if epoch >= 100 and epoch <= 200:
        x = 1e-4
    return x

decay = LearningRateScheduler(step_decay, verbose=1)

optimizer = tf.keras.optimizers.Adam()
tn_model.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])
batch = 32
hist = tn_model.fit(x_train_1d, y_train, 
             batch_size=batch, epochs=300, 
             verbose=1, shuffle=True, 
             steps_per_epoch=int(60000 / batch), callbacks=[decay])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/300
1875/1875 [==============================] - 170s 91ms/step - loss: 2.0786 - accuracy: 0.1019

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 2/300
1875/1875 [==============================] - 167s 89ms/step - loss: 2.0098 - accuracy: 0.1020

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 3/300
1875/1875 [==============================] - 168s 90ms/step - loss: 1.9897 - accuracy: 0.1022

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 4/300
1875/1875 [==============================] - 166s 89ms/step - loss: 1.9743 - accuracy: 0.1024

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 5/300
1875/1875 [==============================] - 169s 90ms/step - loss: 1.9680 - accuracy: 0.1032

Epoch 00006: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 6/300
1875/1875 [===========================

1875/1875 [==============================] - 160s 85ms/step - loss: 1.8870 - accuracy: 0.1601

Epoch 00048: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 48/300
1875/1875 [==============================] - 160s 85ms/step - loss: 1.8755 - accuracy: 0.1599

Epoch 00049: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 49/300
1875/1875 [==============================] - 161s 86ms/step - loss: 1.8673 - accuracy: 0.1613

Epoch 00050: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 50/300
1875/1875 [==============================] - 159s 85ms/step - loss: 1.8654 - accuracy: 0.1623

Epoch 00051: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 51/300
1875/1875 [==============================] - 160s 85ms/step - loss: 1.8672 - accuracy: 0.1643

Epoch 00052: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 52/300
1875/1875 [==============================] - 158s 84ms/step - loss: 1.8770 - accuracy: 0.1643

Epoch 00053: Learnin

1875/1875 [==============================] - 161s 86ms/step - loss: 2.3026 - accuracy: 0.1781

Epoch 00094: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 94/300
1875/1875 [==============================] - 162s 87ms/step - loss: 2.3106 - accuracy: 0.1930

Epoch 00095: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 95/300
1875/1875 [==============================] - 158s 84ms/step - loss: 2.3026 - accuracy: 0.2074

Epoch 00096: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 96/300
1875/1875 [==============================] - 161s 86ms/step - loss: 2.3026 - accuracy: 0.2011

Epoch 00097: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 97/300
1875/1875 [==============================] - 162s 86ms/step - loss: 2.3026 - accuracy: 0.1895

Epoch 00098: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 98/300
1875/1875 [==============================] - 162s 86ms/step - loss: 2.4224 - accuracy: 0.1789

Epoch 00099: Learnin

1875/1875 [==============================] - 160s 85ms/step - loss: 2.2999 - accuracy: 0.1656

Epoch 00140: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 140/300
1875/1875 [==============================] - 161s 86ms/step - loss: 2.3038 - accuracy: 0.1507

Epoch 00141: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 141/300
1875/1875 [==============================] - 159s 85ms/step - loss: 2.3023 - accuracy: 0.1474

Epoch 00142: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 142/300
1875/1875 [==============================] - 160s 85ms/step - loss: 2.3017 - accuracy: 0.1630

Epoch 00143: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 143/300
1875/1875 [==============================] - 158s 85ms/step - loss: 2.3406 - accuracy: 0.1762

Epoch 00144: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 144/300
1875/1875 [==============================] - 158s 84ms/step - loss: 2.2943 - accuracy: 0.1838

Epoch 00145: Le

KeyboardInterrupt: 

In [13]:
label_len = 1
label_num = 10
dim = 2
bond_dim = 10

tf.keras.backend.set_floatx('float64')

tn_model2 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(28, 28, 1)),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        TNLayer(196, label_num, bond_dim),
        tf.keras.layers.Softmax()
    ],name="sequential_1")

tn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d_2 (Average (None, 14, 14, 1)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 196)               0         
_________________________________________________________________
tn_layer_2 (TNLayer)         (None, 10)                39840     
_________________________________________________________________
softmax_2 (Softmax)          (None, 10)                0         
Total params: 39,840
Trainable params: 39,840
Non-trainable params: 0
_________________________________________________________________


In [14]:
x_train_2d = x_train[:, :, :, tf.newaxis]

def step_decay(epoch):
    x = 1e-4
    if epoch >= 100 and epoch <= 200:
        x = 1e-5
    return x

decay = LearningRateScheduler(step_decay, verbose=1)
batch = 32

optimizer = tf.keras.optimizers.Adam()
tn_model2.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])
history = tn_model2.fit(x_train_2d, y_train, batch_size=batch, shuffle=True, 
             steps_per_epoch=int(60000 / batch), epochs=300, verbose=1, callbacks=[decay])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/300
1875/1875 [==============================] - 51s 27ms/step - loss: 2.0648 - accuracy: 0.1188

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 2/300
1875/1875 [==============================] - 49s 26ms/step - loss: 1.9590 - accuracy: 0.1705

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 3/300
1875/1875 [==============================] - 49s 26ms/step - loss: 1.8918 - accuracy: 0.2611

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 4/300
1875/1875 [==============================] - 51s 27ms/step - loss: 1.8569 - accuracy: 0.2684

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 5/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.7742 - accuracy: 0.3078

Epoch 00006: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 6/300
1875/1875 [==============================] 

1875/1875 [==============================] - 50s 27ms/step - loss: 1.0149 - accuracy: 0.6535

Epoch 00048: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 48/300
1875/1875 [==============================] - 50s 27ms/step - loss: 0.9815 - accuracy: 0.6645

Epoch 00049: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 49/300
1875/1875 [==============================] - 50s 27ms/step - loss: 0.9643 - accuracy: 0.6693

Epoch 00050: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 50/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9523 - accuracy: 0.6742

Epoch 00051: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 51/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9441 - accuracy: 0.6761

Epoch 00052: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 52/300
1875/1875 [==============================] - 50s 27ms/step - loss: 0.9360 - accuracy: 0.6797

Epoch 00053: LearningRateS

KeyboardInterrupt: 

In [16]:
x_test_2d = x_test[:, :, :, tf.newaxis]
pred = tn_model2.predict(x_test_2d)

In [19]:
pred[1]

array([9.88396434e-05, 5.02506551e-05, 2.41412798e-01, 1.84134023e-02,
       2.93382341e-05, 6.80038160e-02, 6.71760264e-01, 1.64887619e-07,
       1.26278551e-04, 1.04848077e-04])

In [20]:
test_loss, test_acc = tn_model2.evaluate(x_test_2d, y_test, verbose=1)

313/313 [==============================] - 8s 24ms/step - loss: 0.9037 - accuracy: 0.7687


In [26]:
from tensorflow.keras.models import load_model

tn_model2.save('./TN_MNIST_model_lr5em4_batch64.h5')
#del model
#model = load_model('/path/to/model.h5')

NotImplementedError: Layer TNLayer has arguments in `__init__` and therefore must override `get_config`.